The following code snippet is authored by:<br>
- Markus Fath https://github.com/fath-markus

In [ ]:
# !pip install pandas
# !pip install hana_ml
# !pip install generative-ai-hub-sdk

In this notebook, we will run a basic Q&A using Retrieval Augmented Generation.<br>
The context is retrieved via a vector search in HANA Cloud.<br>
First, we import some data from a csv file (which contains vector embeddings generated by text-embedding-ada-002).<br>
Then we define a run_vector_search function which is is used in the ask_llm function to provide context which is infused into the prompt.<br>

You need to set up SAP AI Core and generate "deployments" for the services you want to use.

Import some vector data from csv file

In [1]:
# import some vector data from csv
import pandas as pd
df = pd.read_csv('./data/GRAPH_DOCU_QRC3.csv', low_memory=False)
df.head(3)

,ID,L1,L2,L3,FILENAME,HEADER1,HEADER2,TEXT,VECTOR_STR
0,273,90,40,0,090-040-000-Appendix_C_-_GraphScript_Cheat_She...,Appendix C - GraphScript Cheat Sheet,Weighted Path Functions,<!--! subsection -->\n### WEIGHT \n```graphsc...,"[0.015699435,0.020284351,0.0003677337,-0.00413..."
1,52,60,20,30,060-020-030-Basic_Vertex_Operations.md,Basic Vertex Operations,DEGREE,Returns the number of incoming and outgoing ed...,"[0.018821003,0.012627394,-0.007940338,-0.00959..."
2,44,60,20,20,060-020-020-Basic_Graph_Operations.md,Basic Graph Operations,EDGES,Returns all edges in a graph. \n- EDGES(GRAPH...,"[-0.013607875,0.009249507,-0.03403819,-0.03394..."


In [2]:
# connect to HANA
from hana_ml.dataframe import ConnectionContext
# cc = ConnectionContext(userkey='GR3', encrypt=True)
cc= ConnectionContext(
    address='[somehost].hanacloud.ondemand.com', 
    port='443', 
    user='[your user]', 
    password='[your password]', 
    encrypt=True
    )
print(cc.hana_version())
print(cc.get_current_schema())

4.00.000.00.1708429435 (fa/CE2024.2)
GRAPH_USER


In [4]:
# # Drop table if already exists
# cursor = cc.connection.cursor()
# sql_command = '''DROP TABLE GRAPH_DOCU_QRC3;'''
# cursor.execute(sql_command)
# cursor.close()

In [5]:
# Create a table
cursor = cc.connection.cursor()
sql_command = '''CREATE TABLE GRAPH_DOCU_QRC3(ID BIGINT, L1 NVARCHAR(3), L2 NVARCHAR(3), L3 NVARCHAR(3), FILENAME NVARCHAR(100), HEADER1 NVARCHAR(5000), HEADER2 NVARCHAR(5000), TEXT NCLOB, VECTOR_STR NCLOB);'''
cursor.execute(sql_command)
cursor.close()

In [6]:
# Upload data
from hana_ml.dataframe import create_dataframe_from_pandas
v_hdf = create_dataframe_from_pandas(
    connection_context=cc,
    pandas_df=df,
    table_name="GRAPH_DOCU_QRC3", 
    allow_bigint=True,
    append=True
    )

100%|██████████| 1/1 [00:02<00:00,  2.94s/it]


In [7]:
# Add REAL_VECTOR column
cursor = cc.connection.cursor()
sql_command = '''ALTER TABLE GRAPH_DOCU_QRC3 ADD (VECTOR REAL_VECTOR(1536));'''
cursor.execute(sql_command)
cursor.close()

In [8]:
# Create vectors from strings
cursor = cc.connection.cursor()
sql_command = '''UPDATE GRAPH_DOCU_QRC3 SET VECTOR = TO_REAL_VECTOR(VECTOR_STR);'''
cursor.execute(sql_command)
cursor.close()

RAG using gen ai hub sdk and HANA Vector Search

In [9]:
# Get embeddings
from gen_ai_hub.proxy.native.openai import embeddings

def get_embedding(
    input, model_name="text-embedding-ada-002") -> str:
    response = embeddings.create(
        input=input,
        model_name=model_name, 
        )
    return response.data[0].embedding

In [10]:
# Wrapping HANA vector search in a function
def run_vector_search(query: str, metric="COSINE_SIMILARITY", k=4):
    if metric == 'L2DISTANCE':
        sort = 'ASC'
    else:
        sort = 'DESC'
    query_vector = get_embedding(query)
    sql = '''SELECT TOP {k} "ID", "HEADER1", "HEADER2", "TEXT"
        FROM "GRAPH_DOCU_QRC3"
        ORDER BY "{metric}"("VECTOR", TO_REAL_VECTOR('{qv}')) {sort}'''.format(k=k, metric=metric, qv=query_vector, sort=sort)
    hdf = cc.sql(sql)
    df_context = hdf.head(k).collect()
    # context = ' '.join(df_context['TEXT'].astype('string'))
    return df_context

In [11]:
# Test the vector search
query = "How can I run a shortest path algorithm?"
df_context = run_vector_search(query=query, metric="COSINE_SIMILARITY",k=4)
df_context

,ID,HEADER1,HEADER2,TEXT
0,211,Complex GraphScript Examples,GraphScript Procedure Example,The following example depicts a more complex e...
1,90,Graph Traversal Statements,Dijkstra's Algorithm (DIJKSTRA),DIJKSTRA searches for shortest paths in a weig...
2,83,Built-In Graph Algorithms,Shortest Path,```bnf\n<sssp_function> ::= SHORTEST_PATH '(' ...
3,65,Basic Weighted Path Operations,(Constructors),WEIGHTEDPATH objects can’t be constructed dire...


In [18]:
from gen_ai_hub.proxy.native.openai import chat

# basic LLM prompt for RAG
sys_content = '''Your task is to answer the question using the provided context wrapped in triple quotes. 
If the provided context does not contain the information needed to answer this question then come up with your own answer. '''

def ask_llm(query: str, retrieval_augmented_generation: bool, metric='COSINE_SIMILARITY', k = 4) -> str:
    context = ''
    if retrieval_augmented_generation == True:
        df_context = run_vector_search(query, metric, k)
        context = ' '.join(df_context['TEXT'].astype('string'))
    user_content = '"""' + context + '"""'+ ' Question: ' + query
    messages=[{"role": "system", "content": sys_content}, {"role": "user", "content": user_content}]
    
    # kwargs = dict(model_name='gpt-4', messages=messages)
    kwargs = dict(model_name='gpt-35-turbo', messages=messages)
    
    response = chat.completions.create(**kwargs)
    
    return response

In [21]:
# query = "Can you define a HANA graph workspace on a JSON document store collection?"
#query = "How can I define a HANA graph workspace on a JSON document store collection?"
#query = "How do you run a shortest path algorithm in SAP HANA Graph engine?"
# query = "How can I run community detection Louvain in SAP HANA Graph?"
# query = "How can I run a BFS traversal in HANA Cloud"
query = "I want to calculate a shortest path. How do I do that?"

response = ask_llm(query=query, retrieval_augmented_generation=True, k=4)

print(response.choices[0].message.content)

To calculate a shortest path, you can use the SHORTEST_PATH function in GraphScript. Here is the syntax for the SHORTEST_PATH function:
```
SHORTEST_PATH(<parent_graph_variable>, <start_vertex>, <target_vertex> [, <clos_expr>] [, <edge_direction>])
```
- `<parent_graph_variable>`: The variable reference for the graph.
- `<start_vertex>`: The expression for the start vertex.
- `<target_vertex>`: The expression for the target vertex.
- `<clos_expr>` (optional): A closure expression that defines a custom weight function.
- `<edge_direction>` (optional): The expression for the edge direction. Possible values are 'OUTGOING', 'INCOMING', and 'ANY'.

The SHORTEST_PATH function returns a WEIGHTEDPATH instance, which represents the shortest path within the given parent graph from the start vertex to the target vertex. If no path exists, an empty path is returned.

Here is an example of using the SHORTEST_PATH function to calculate the shortest path between two vertices:
```graphscript
GRAPH g =